In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("/scratch/ewalt/pdm/rs-uncertainty")
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from src.metrics import StratifiedRCU
from src.viz import *
from pathlib import Path
import rasterio
import fiona
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import trange
import random
import yaml
import fiona
import rasterio.warp
import rasterio.features
sns.set()
sns.set_style("whitegrid")
random.seed(123)

RESDIR = "results/cloud_exp/2023-06-20_16-14-11" #"results/cloud_exp/2023-05-26_15-55-46"
S2DIR = "gee_data/reprojected/"
S2REPRDIR = "gee_data/reprojected_dirs"
GTDIR = "assets/data/preprocessed"
SANITYRESDIR = "results/cloud_exp/2023-05-31_11-23-56_sanity_check" # results
SANITYS2DIR = "assets/data/sentinel_data/s2_reprojected" # s2 reprojected
SANITYS2REPRDIR = "gee_data/sanity_check/" # restructured s2 reprojected
SPLITMASKDIR = "assets/data/split_masks/" # split masks
SHAPEFILES = ['assets/data/NHM_projectDekning_AOI_edit2015_V2.shp', 'assets/data/ALS_projects_Dz_all_norway.shp']
STATSFILE = "data/2023-04-05_18-58-33_baseline/stats.yaml"
EMPIRICAL_CP_THRESHOLD = 7.9

with open(STATSFILE, "r") as f:
    stats = yaml.safe_load(f)
TRAINMEANS = stats["labels_stats"]["mean"]
TRAINSTDS = stats["labels_stats"]["std"]
for i in [2,4]:
    TRAINMEANS[i] /= 100
    TRAINSTDS[i] /= 100
    
VARIABLES = ['P95', 'MeanH', 'Dens', 'Gini', 'Cover']

# Experiment result directories
result_dirs = [p.path for p in os.scandir(RESDIR) if os.path.exists(os.path.join(p.path,"rcu.json"))]
outliers = [os.path.join(RESDIR, f"1023_{d}") for d in [
    "20180503T104019", # index: 3, avgcp: 42, all white
    "20180620T105031", # index: 6, avgcp: 0., all white
]]
result_dirs = [r for r in result_dirs if not any(r.__contains__(o) for o in outliers)]
len(result_dirs)

## Raster-level

In [ ]:
selected_result_dirs = [os.path.join(RESDIR, p) for p in [
 '1023_20180526T105029',
 '1023_20180925T104021',
 '1023_20180918T105021',
 '1023_20180528T104021']]
conditions = ["no cloud", "thin clouds", "cloudy", "very cloudy"]
islice, jslice = slice(470, 600), slice(150, 300)

In [ ]:
u_bounds = [9, 6.5, 0.35, 0.12, 0.42] # like other maps in report
std_bounds = [0,0,0,0,0], u_bounds
aleatoric_diff_bounds = [7.5, 5, 0.3, 0.10, 0.3]
epistemic_diff_bounds = [7.5, 5, 0.3, 0.10, 0.3]
kwargs = dict(
    aleatoric_diff_bounds=aleatoric_diff_bounds,
    epistemic_diff_bounds=epistemic_diff_bounds,
    std_bounds=std_bounds,
)
for varname in VARIABLES:
    varindex = VARIABLES.index(varname)+1
    showUncertaintyTypes(
        selected_result_dirs,
        conditions,
        varindex,
        VARIABLES[i],
        S2REPRDIR,
        islice=islice,
        jslice=jslice,
        normalize=False,
        save_name=f"images/cloud_experiment/utype_maps_{VARIABLES[i]}",
        **kwargs
    )

In [ ]:
makeConditionUncertaintyTypePlot(
    selected_result_dirs,
    titles=conditions,
    variable_names=VARIABLES,
    islice=islice,
    jslice=jslice,
    save_name="images/cloud_experiment/conditions_uncertainty_type"
)

In [ ]:
makeConditionUncertaintyCoveragePlot(
    selected_result_dirs,
    titles=conditions,
    trainset_means=TRAINMEANS,
    variable_names=VARIABLES,
    islice=islice,
    jslice=jslice,
    save_name="images/cloud_experiment/conditions_uncertainty_coverage"
)

## Pixel-level

In [ ]:
visualizer, nnz_result_dirs, exp_vars = get_nonzero_avg_cp_visualizer(
    result_dirs, S2REPRDIR, VARIABLES
)

In [ ]:
# report
for var in VARIABLES:
    showCloudVsUncertaintyType(
        nnz_result_dirs,
        S2REPRDIR,
        VARIABLES.index(var)+1,
        [var],
        figsize=(15,3.75),
        save_name=f"images/cloud_experiment/uncertainty_da_{var}"
    )
    showCloudVsMembersUncertainty(
        nnz_result_dirs,
        S2REPRDIR,
        VARIABLES.index(var)+1,
        [var],
        save_name=f"images/cloud_experiment/uncertainty_aleatoric_{var}",
        figsize=(15,4.5)
    )
    showCloudVsMembersEpistemic(
        nnz_result_dirs,
        S2REPRDIR,
        VARIABLES.index(var)+1,
        [var],
        save_name=f"images/cloud_experiment/uncertainty_epistemic_{var}",
        figsize=(15,4.5)
    )

In [ ]:
# defense
from src.viz import *
for var in VARIABLES:
    print(var)
    singlePlotsCloudVsUncertaintyType(
        nnz_result_dirs,
        S2REPRDIR,
        VARIABLES.index(var)+1,
        [var],
        figsize=(6.7,5.3),
        save_name=f"images/defense/uncertainty_da_{var}"
    )
    singlePlotsCloudVsMembersUncertainty(
        nnz_result_dirs,
        S2REPRDIR,
        VARIABLES.index(var)+1,
        [var],
        save_name=f"images/defense/uncertainty_aleatoric_{var}",
        figsize=(6.7,5.3)
    )
    singlePlotsCloudVsMembersEpistemic(
        nnz_result_dirs,
        S2REPRDIR,
        VARIABLES.index(var)+1,
        [var],
        save_name=f"images/defense/uncertainty_epistemic_{var}",
        figsize=(6.7,5.3)
    )